In [5]:
%reload_ext autoreload
%autoreload 3
%matplotlib inline

import os
import numpy
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt

# Return two paths from the current working directory
current = os.getcwd()
parent_1 = os.path.dirname(current)
parent_2 = os.path.dirname(parent_1)
root_dir = os.path.join(parent_2, 'interface')
# Add the root directory to the system path
os.sys.path.append(root_dir)
# H:\github\nnc-ufmg\micromap\src\micromap\interface\micromap_utils.py
from micromap_utils import MicroMAPReader

In [2]:
def select_folder():
    # Create and force a foreground hidden root window
    root = tk.Tk()
    root.withdraw()
    root.lift()                  # Raise the window
    root.attributes('-topmost', True)  # Keep it on top

    # Show the folder picker
    folder = filedialog.askdirectory(title="Select the recording folder", parent=root)

    # Destroy everything cleanly
    root.update()
    root.destroy()

    if folder:
        print(f"Selected folder: {folder}")
    else:
        print("No folder selected.")

    return folder

In [35]:
%matplotlib qt
import os

folders = select_folder()

total_samples = 0
total_loss = 0
report_file = os.path.join(folders, 'report.txt')

for folder in os.listdir(folders):    
    data_folder = os.path.join(folders, folder)

    if os.path.isdir(data_folder):
        reader = MicroMAPReader(data_folder, 'n')

        with open(report_file, 'a') as report:
            report.write("MicroMAP Data Report\n")
            report.write("====================\n")
            report.write(f"Folder: {data_folder}\n")
            report.write("\n")
            report.write(f"Number of channels: {reader.num_channels}\n")
            report.write(f"Sampling rate: {reader.sampling_freq}\n")
            report.write(f"Number of samples lost: {reader.packets_lost}\n")
            report.write("\n")

    total_samples += reader.data.shape[1]
    total_loss += reader.packets_lost

    arduino_errors = reader.check_arduino_test()
    counter_check = reader.check_packet_counter()

    with open(report_file, 'a') as report:
        report.write("Arduino test results:\n")
        report.write("=====================\n")
        report.write(f"Arduino signal test: {arduino_errors}\n")

        report.write("Packet counter test results:\n")
        report.write("============================\n")
        if counter_check:
            report.write("Counter test passed.\n")
        else:
            report.write("Counter test failed.\n")

    # fig, ax = plt.subplots()
    # time_vector = reader.get_time_vector()
    # data = reader.get_data()

    # for i in range(0, 32):
    #     ax.plot(time_vector[:], data[i, :]/1000 + i, color = 'black', lw = 0.5)

    # ax.set_xlabel("Time (s)")
    # ax.set_ylabel("Channels")
    # ax.set_title("MicroMAP Data")

    # plt.show()

print('---------------------------------------------')
print('Total samples:', total_samples)
print('Total packets lost:', total_loss)

with open(report_file, 'a') as report:
    report.write("\n\n\nTotal samples: " + str(total_samples) + "\n")
    report.write("Total packets lost: " + str(total_loss) + "\n")
    report.write("\n")
    report.write("End of report.\n")

Selected folder: C:/Users/mcjpe/Desktop/micromap_test/spi_test_pc
Packet counter error between 114588 and 114590.
Packet counter error between 116276 and 116278.
Packet counter error between 114590 and 114590.
Packet counter error between 82746 and 82748.
Packet counter error between 2784346 and 2784349.
Packet counter error between 82748 and 82748.
---------------------------------------------
Total samples: 14399235
Total packets lost: 5


In [36]:
obj = reader
data = obj.data[:, 1000:1050]
rate = obj.sampling_freq
  
num_channels = data.shape[0]                                                                                       
time = numpy.arange(data.shape[1])                                                                                 # Creates a vector with the number of samples lenght
time = time /rate                                                                                                   # Converts the samples to seconds

fig = plt.figure(figsize=(10, 5))                                                                                 # Creates a figure with the size of 10x5 inches
ax_0 = fig.add_subplot(1, 1, 1)                                                                                  # Creates a 1x3 matrix of subplots

plot_channels = range(32)

for ch in plot_channels:                                                                                             # Loop through all the rows of the matrix
    ax_0.plot(time, data[ch]*0.001 + ch + 1, color = (160/255, 17/255, 8/255, 1), linewidth = 1)

ax_0.set_xlabel('TIME (s)', fontsize = 16)
ax_0.set_ylabel('CHANNEL', fontsize = 16)
ax_0.set_yticks(range(1, max(plot_channels) + 2, 2))
ax_0.set_yticklabels(range(1, max(plot_channels) + 2, 2))
ax_0.set_xlim(0, time[-1])

# set tick labels font
for label in (ax_0.get_xticklabels() + ax_0.get_yticklabels()):
    label.set_fontsize(16)

plt.savefig(folders + '/signal_example.png')
plt.savefig(folders + '/signal_example.svg')

plt.show()